In [1]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
from common.functions import *
from common.gradient import numerical_gradient
import numpy as np


In [6]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 重み,バイアスの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    #　各層のニューロンの計算　認識 推論
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # x:入力データ, t:教師データ
    # 教師データと照らし合わせて、整合性を評価
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t) # 損失関数で値を算出
    
    # 単純に精度を算出
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x:入力データ, t:教師データ
    # 勾配を算出
    def numerical_gradient(self, x, t):
        # エントロピー誤差計算からでた誤差
        loss_W = lambda W: self.loss(x, t)
       # print(loss_W)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    # numerical_gradientの高速版 
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        dz1 = np.dot(dy, W2.T)
        da1 = sigmoid_grad(a1) * dz1
        grads['W1'] = np.dot(x.T, da1)
        grads['b1'] = np.sum(da1, axis=0)

        return grads


In [7]:
net = TwoLayerNet(input_size = 784, hidden_size = 100, output_size = 10)

In [8]:
x = np.random.rand(100,784) # 画像データ
y  = net.predict(x)
#print(y)

In [10]:
t = np.random.rand(100,10) # 教師データ
grads = net.gradient(x, t)
#print(loss_W)
print(grads)
#print(len(grads))

{'W2': array([[-0.21834379, -0.17754676, -0.19862208, -0.17716368, -0.20605108,
        -0.19059327, -0.21429483, -0.20670069, -0.22228236, -0.22243474],
       [-0.22007631, -0.17923766, -0.19863625, -0.17629938, -0.20676212,
        -0.1895457 , -0.21372129, -0.2069075 , -0.22211216, -0.22312198],
       [-0.22407945, -0.18307995, -0.20486456, -0.18054272, -0.20986282,
        -0.19558684, -0.21875303, -0.21166991, -0.22933771, -0.22818384],
       [-0.21808371, -0.17758381, -0.19649777, -0.17582272, -0.20416567,
        -0.18850038, -0.21224268, -0.20562628, -0.22167653, -0.22074679],
       [-0.19260348, -0.1560839 , -0.1756818 , -0.15569788, -0.1809219 ,
        -0.16750839, -0.18821612, -0.18257914, -0.19677414, -0.19632533],
       [-0.20823527, -0.16879517, -0.18667582, -0.16772037, -0.19448168,
        -0.18079294, -0.20253047, -0.19589392, -0.20932688, -0.21119342],
       [-0.19978803, -0.16302683, -0.18177948, -0.16179711, -0.18835881,
        -0.1742497 , -0.19499009, -0.1

In [ ]:
print(grads['W1'].shape)
print(grads['b1'].shape)
print(grads['W2'].shape)
print(grads['b2'].shape)